## Load PyEpiDAG

In [1]:
import epidag as dag
import numpy as np

## Define a DAG

Compose a script

```
PCore Exp1 {
    # Definitions of nodes
}
```

Then, .. parse to a DAG

In [2]:
script = '''
PCore Exp1 {
    n
    a = 0.5 
    b ~ beta(1, 1)
    c = min(a, b)
    y ~ binom(n, c)
}
'''

js = dag.bn_script_to_json(script)
bn = dag.BayesianNetwork(js)
bn

Name:	Exp1
Nodes:
	n
	b ~ beta(1,1)
	a = 0.5
	c = min(a,b)
	y ~ binom(n,c)

### Single value variable


> VariableName = Number 


In [3]:
SingleValue = bn['a']
print('Node \'a\'')
print(SingleValue)

print('\nFind the value')
print(SingleValue())

Node 'a'
a = 0.5

Find the value
0.5


### Exogenous variable

> VariableName

In [4]:
Exogenous = bn['n']
print('Node \'n\'')
print(Exogenous)

print('\nFind the value; must append external resources')
print(Exogenous({'n': 5}))

Node 'n'
n

Find the value; must append external resources
5


### Random variable

> VariableName ~ p(...)

** p(...) **: a probabilidy density/mass function

In [5]:
Random = bn['b']
print('Node \'b\'')
print(Random)

print('\nSample a value')
print(Random())

Node 'b'
b ~ beta(1,1)

Sample a value
0.732622402088


### Equation output

> OutputName = g(...)

** g(...) **: a mathematical function

In [6]:
Equation = bn['c']
print('Node \'c\'')
print(Equation)

parents = {
    'a': SingleValue(), 
    'b': Random() 
}

print('\nCalculate a value')
print(Equation(parents))

Node 'c'
c = min(a,b)

Calculate a value
0.282966987657


### Pseudo Node
Pseudo nodes are the nodes which can not be implemented in simulation but in relation inference.

> VariableName = f(...)

** f(...) **: a pseudo function start with ** f ** and a list of parent variables follows

In [7]:
## todo

## For simulation model

### Basic case

In [8]:
script = '''
PCore Exp2 {
   mu_x = 0
   mu_y = 10
   sd ~ exp(1)
   x ~ norm(mu_x, sd)
   y ~ norm(mu_y, sd)
}
'''

bn = dag.bn_from_script(script)
sc = dag.as_simulation_core(bn)
sg = sc.generate('exp2')
sg

sd: 1.88104, mu_y: 10, mu_x: 0

In [9]:
y = sg.get_sampler('y')
y

Actor y (norm(mu_y,sd)) on exp2

In [10]:
y(), np.mean([y() for _ in range(1000)])

(9.5761431222865472, 9.9412444283247225)

Intervention

In [11]:
sg.impulse({'mu_y': 100})
sg

{'y'}
{'y'}


sd: 1.88104, mu_y: 100, mu_x: 0

In [12]:
y(), np.mean([y() for _ in range(1000)])

(100.54780045181155, 100.00488304975876)

### Define exposed nodes

### Heirarchical node groups

### Random effects

### Example 1: beta-binomial model

In [13]:
# Sample 

In [14]:
# Sample with given nodes

### Example 2:  regression model

### Example 3: BMI model

In [15]:

script = '''
Pcore BMI {
    b0 ~ norm(12, 1)
    b1 = 0.5 # gggg
    pf ~ beta(8, 20)
    foodstore ~ binom(100, pf)
    b0r ~ norm(0, .01)
    ageA ~ norm(20, 3)
    ageB ~ norm(30, 2)
    ps ~ beta(5, 6)
    sexA ~ cat({'m': ps, 'f': 1-ps})
    muA = b0 + b0r + b1*ageA
    bmiA ~ norm(muA, sd)
    sdB = sd * 0.5
    muB = b0 + b0r + b1*ageB
    bmiB ~ norm(muB, sdB)
}
'''


bn = dag.bn_from_script(script)

hie = {
    'country': ['area'],
    'area': ['b0r', 'ps', 'foodstore', 'agA', 'agB'],
    'agA': ['bmiA', 'ageA', 'sexA'],
    'agB': ['bmiB', 'ageB']
}

sc = dag.as_simulation_core(bn, hie,
                            root='country',
                            random=['muA'],
                            out=['foodstore', 'bmiA', 'bmiB'])

pc = sc.generate('Taiwan', {'sd': 1})
pc_taipei = pc.breed('Taipei', 'area')
pc_taipei.breed('A1', 'agA')
pc_taipei.breed('A2', 'agA')
pc_taipei.breed('B1', 'agB')
pc_taipei.breed('B2', 'agB')

sd: 1, sdB: 0.5, pf: 0.226975, b1: 0.5, b0: 12.8864, ps: 0.365611, b0r: 0.00982017, ageB: 29.5779, muB: 27.6852

In [16]:
pc.deep_print()

Taiwan (sd: 1, sdB: 0.5, pf: 0.226975, b1: 0.5, b0: 12.8864)
-- Taipei (sd: 1, sdB: 0.5, pf: 0.226975, b1: 0.5, b0: 12.8864, ps: 0.365611, b0r: 0.00982017)
---- A1 (sd: 1, sdB: 0.5, pf: 0.226975, b1: 0.5, b0: 12.8864, ps: 0.365611, b0r: 0.00982017, sexA: m, ageA: 22.4131)
---- A2 (sd: 1, sdB: 0.5, pf: 0.226975, b1: 0.5, b0: 12.8864, ps: 0.365611, b0r: 0.00982017, sexA: m, ageA: 16.1108)
---- B1 (sd: 1, sdB: 0.5, pf: 0.226975, b1: 0.5, b0: 12.8864, ps: 0.365611, b0r: 0.00982017, ageB: 27.6241, muB: 26.7083)
---- B2 (sd: 1, sdB: 0.5, pf: 0.226975, b1: 0.5, b0: 12.8864, ps: 0.365611, b0r: 0.00982017, ageB: 29.5779, muB: 27.6852)


In [20]:
pc.impulse({'b0r': 1})
pc.deep_print()

{'bmiB', 'bmiA', 'muB', 'muA'}
{'bmiB', 'bmiA', 'muB', 'muA'}
Taiwan (sd: 1, sdB: 0.5, pf: 0.226975, b1: 1, b0: 1)
-- Taipei (sd: 1, sdB: 0.5, pf: 0.226975, b1: 0.5, b0: 12.8864, ps: 0.365611, b0r: 1)
---- A1 (sd: 1, sdB: 0.5, pf: 0.226975, b1: 0.5, b0: 12.8864, ps: 0.365611, b0r: 0.00982017, sexA: m, ageA: 22.4131)
---- A2 (sd: 1, sdB: 0.5, pf: 0.226975, b1: 0.5, b0: 12.8864, ps: 0.365611, b0r: 0.00982017, sexA: m, ageA: 16.1108)
---- B1 (sd: 1, sdB: 0.5, pf: 0.226975, b1: 0.5, b0: 12.8864, ps: 0.365611, b0r: 0.00982017, ageB: 27.6241, muB: 26.7083)
---- B2 (sd: 1, sdB: 0.5, pf: 0.226975, b1: 0.5, b0: 12.8864, ps: 0.365611, b0r: 0.00982017, ageB: 29.5779, muB: 27.6852)


In [18]:
pc_taipei.Locus

{'b0': 12.886403793104588,
 'b0r': 0.0098201740251844388,
 'b1': 0.5,
 'pf': 0.22697480441178802,
 'ps': 0.36561100342119301,
 'sd': 1,
 'sdB': 0.5}

### Example 4: A simple agent-based model 